In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import plotly.graph_objects as go
from matplotlib import pyplot as plt
import numpy as np

import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def adaboost(X, y, n_estimators=10):

    stumps = []
    alphas = []
    errors = []
    ada = []

    shape = X.shape[0]
    weights = np.zeros(shape=(n_estimators, shape))

    #Inicialização dos pesos como sendo 1/m
    weights[0] = [1/shape] *shape
    
    for i in range(n_estimators):

        # stump
        stump = DecisionTreeClassifier(max_depth=1)
        stump.fit(X, y, sample_weight=weights[i])
        pred = stump.predict(X)

        #Calculo do erro
        error = sum(weights[i][(pred != y)])

        #Calculando alpha t
        alpha = 1/2 * np.log((1 - error) / error) 

        # Atualização dos pesos
        new_weights = (weights[i] * np.exp(-alpha * y * pred))
        new_weights = new_weights/new_weights.sum()

        if i+1 < n_estimators:
            weights[i+1] = new_weights

        stumps.append(stump)
        errors.append(error)
        alphas.append(alpha)
        ada.append(np.prod(((errors[i]*(1-errors[i]))**1/2)))

    return ada, errors, stumps, alphas

In [ ]:
def predict(X, alphas, stumps):
    return np.sign(np.dot(alphas, np.array([stump.predict(X) for stump in stumps])))